In [1]:
#imports

import time
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import random
import pandas as pd

In [6]:
# Configure Selenium
chrome_options = webdriver.ChromeOptions()
loc = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_options.binary_location = loc

# Initialize the WebDriver with options
driver = webdriver.Chrome(options=chrome_options)

# Initialize a list to store the scraped data
data_list = []

# Keep track of visited URLs
visited_urls = set()

# Function to find elements from soup and append it to data_list
def find_and_append(data_dict, field_name, xpath):
    try:
        data_element = driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        data_element = " "
    data_dict[field_name] = data_element
    
# Search term and initial page URL
search_term = "laptop"
base_url = "https://www.amazon.com/"

driver.get(base_url)

# Wait for the search input element to be present and clickable
wait = WebDriverWait(driver, 10)
search_input = wait.until(EC.element_to_be_clickable((By.ID, "twotabsearchtextbox")))

search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)

# Loop through multiple search pages
for page in range(1,10):
    
    # Wait time
    time.sleep(3)
    
    # Parse the page w BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Find links to 'search_term' pages
    product_links = soup.find_all("a", class_="a-link-normal", href=True)
    
    next_page_links = soup.find_all("a", class_="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator", href=True)

    next_link_href = next_page_links[0].get("href")
    # Empty list to store href
    href_list = []
    
    for product_link in product_links:
        href = product_link.get("href")
        
        if (
            href.startswith("/sspa") or
            href.endswith("#customerReviews") or
            href.startswith("https") or
            href.startswith("/s") or
            href.startswith("#") or
            href.startswith("/gp") or
            "/ref=v" in href
        ):
            continue
        else:
            href_list.append(href)
            
    for x in range(len(href_list)):
        href = href_list[x]
        
        if href not in visited_urls:
            print(href)

            # Direct driver to the page
            driver.get("https://www.amazon.com"+href)
            
            data_dict = {}
            
            try:
                product_price = driver.find_element(By.XPATH, '//*[@id="priceblock_ourprice"]').text
            except NoSuchElementException:
                    try:
                        product_price = driver.find_element(By.XPATH, '//*[@id="priceblock_dealprice"]').text
                    except NoSuchElementException:
                        try:
                            product_price = driver.find_element(By.XPATH, '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[1]/span[2]/span[2]').text
                        except NoSuchElementException:
                            product_price = " "
                            
            find_and_append(data_dict, 'product_title', '//*[@id="productTitle"]')
            find_and_append(data_dict, 'product_brand','//*[@id="poExpander"]/div[1]/div/table/tbody/tr[1]/td[2]/span')
            find_and_append(data_dict, 'model_name', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[2]/td[2]/span')
            find_and_append(data_dict, 'screen_Size', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[3]/td[2]/span')
            find_and_append(data_dict, 'product_color', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[4]/td[2]/span')
            find_and_append(data_dict, 'harddisk_size', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[5]/td[2]/span')
            find_and_append(data_dict, 'cpu_model', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[6]/td[2]/span')
            find_and_append(data_dict, 'ram_size', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[7]/td[2]/span')
            find_and_append(data_dict, 'operating_system', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[8]/td[2]/span')
            find_and_append(data_dict,'graphic_card', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[10]/td[2]/span')
            find_and_append(data_dict, 'graphics_coprocessor', '//*[@id="poExpander"]/div[1]/div/table/tbody/tr[10]/td[2]/span')
            find_and_append(data_dict, 'product_processor', '//*[@id="productDetails_techSpec_section_1"]/tbody/tr[4]/td' )
            
            # Add href to visite urls
            visited_urls.add(href)
            
            data_list.append(data_dict)
            
    current_url = base_url + next_link_href

    driver.get(current_url)
            
driver.quit()

# Create a Pandas DataFrame from the data_list
df = pd.DataFrame(data_list)

# Save the DataFrame to a CSV file
df.to_csv("amazon_laptop_data.csv", index=False, encoding="utf-8")

/ASUS-ROG-Strix-Gaming-Laptop/dp/B0BV7XQ9V9/ref=sr_1_3?keywords=laptop&qid=1694954672&sr=8-3
/Lenovo-IdeaPad-Display-Celeron-Graphics/dp/B0BLW25MB7/ref=sr_1_4?keywords=laptop&qid=1694954672&sr=8-4
/HP-Micro-edge-Microsoft-14-dq0040nr-Snowflake/dp/B0947BJ67M/ref=sr_1_5?keywords=laptop&qid=1694954672&sr=8-5
/Dell-Inspiron-3520-Touchscreen-i7-1255U/dp/B0BN2DM7T5/ref=sr_1_6?keywords=laptop&qid=1694954672&sr=8-6
/ASUS-Chromebook-Processor-Graphics-Speakers/dp/B0C7NMWZNS/ref=sr_1_8?keywords=laptop&qid=1694954672&sr=8-8
/climatepledgefriendly
/ASUS-WQXGA-Touchscreen-Gaming-Laptop/dp/B0CBS19843/ref=sr_1_9?keywords=laptop&qid=1694954672&sr=8-9
/ASUS-IPS-Type-Display-Keyboard-FA706IH-RS53/dp/B0B728K77Q/ref=sr_1_10_mod_primary_new?keywords=laptop&qid=1694954672&sr=8-10
/MSI-Thin-144Hz-Gaming-Laptop/dp/B0BT3CD75G/ref=sr_1_11?keywords=laptop&qid=1694954672&sr=8-11
/LG-Lightweight-Business-Platform-i7-1360P/dp/B0C9DGRYLJ/ref=sr_1_13?keywords=laptop&qid=1694954672&sr=8-13
/ASUS-Vivobook-i5-1135G7-i7-

In [11]:
df.head()

,product_title,product_brand,model_name,screen_Size,product_color,harddisk_size,cpu_model,ram_size,operating_system,graphic_card,graphics_coprocessor,product_processor
0,"ASUS ROG Strix G16 (2023) Gaming Laptop, 16” 1...",ASUS,ROG Strix G16,16 Inches,Eclipse Gray,1 TB,Intel Core i9,16 GB,,,,5.6 GHz intel_core_i9
1,"Lenovo IdeaPad 1 14 Laptop, 14.0"" HD Display, ...",Lenovo,Lenovo Ideapad 1i,14 Inches,Gray,64 GB,Celeron N4020,4 GB,,,,1.1 GHz celeron_n4020
2,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",HP,14-dq0040nr,14 Inches,Snowflake White,64 GB,Celeron N4020,4 GB,,,,2.8 GHz celeron_n4020
3,"Dell Inspiron 15 3000 Series 3530 Laptop, 15.6...",Dell,Inspiron 15 3530,15.6 Inches,Black,1 TB,Core i7 Family,32 GB,,,,1.2 GHz core_i7_family
4,"ASUS C523NA Chromebook 15.6"" HD Laptop, Intel ...",ASUS,15.6 Inches,Silver,64 GB,Celeron N,4 GB,,,,,1.1 GHz celeron_n
